En este notebook se prepararán los datasets que servirán de insumo para las consultas a la API

In [1]:
import pandas as pd
import html

## PlayTimeGenre

**Consigna:**

def PlayTimeGenre(genero : str): Debe devolver año con mas horas jugadas para dicho género.
Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

Algunas consideraciones:
* El **género del juego** está en la columna "genres" del dataset "steam_games_procesado.csv".
* El **año de lanzamiento** está en la columna "year" del dataset "steam_games_procesado.csv".
* El **tiempo de juego** está en la columna "playtime_forever" del dataset "user_items_procesado.csv".
* Podemos vincular ambos datasets a través de la columna **"id"** de "steam_games_procesado.csv" y la columna **"item_id"** de "user_items_procesado.csv".

In [2]:
games_df = pd.read_csv("./Datasets/steam_games_procesado.csv")
games_df = games_df[["id", "year", "genres"]]

games_df.head()

,id,year,genres
0,761140,2018.0,"['Action', 'Casual', 'Indie', 'Simulation', 'S..."
1,643980,2018.0,"['Free to Play', 'Indie', 'RPG', 'Strategy']"
2,670290,2017.0,"['Casual', 'Free to Play', 'Indie', 'Simulatio..."
3,767400,2017.0,"['Action', 'Adventure', 'Casual']"
4,773570,NaN,NaN


In [3]:
games_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32125 entries, 0 to 32124
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      32125 non-null  int64  
 1   year    29955 non-null  float64
 2   genres  28850 non-null  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 753.1+ KB


In [4]:
# Analizar los valores nulos.
games_df.isna().sum()

id           0
year      2170
genres    3275
dtype: int64

Para esta consulta no nos sirven aquellos registros que tengan valores nulos en "year" o "genres", por lo que los eliminaremos.

In [5]:
# Borramos valores nulos
games_df.dropna(subset=["year", "genres"], inplace=True, ignore_index=True)

# Casteamos "year" a entero
games_df["year"] = games_df["year"].astype("int32")

Luego, para poder agrupar por género, necesitaremos que cada género ocupe una fila. Lograremos esto con la función pd.explode().

In [6]:
games_df.shape

(28728, 3)

In [7]:
# Usar eval() para convertir las listas de strings entre comillas en listas de 
# cadenas de texto válidas antes de usar explode().
games_df['genres'] = games_df['genres'].apply(eval)

games_df = games_df.explode("genres")

games_df.reset_index(inplace=True, drop=True) # Reseteamos el indice

games_df.head()

,id,year,genres
0,761140,2018,Action
1,761140,2018,Casual
2,761140,2018,Indie
3,761140,2018,Simulation
4,761140,2018,Strategy


In [8]:
games_df.genres.unique()

array(['Action', 'Casual', 'Indie', 'Simulation', 'Strategy',
       'Free to Play', 'RPG', 'Sports', 'Adventure', 'Racing',
       'Massively Multiplayer', 'Early Access',
       'Animation &amp; Modeling', 'Video Production', 'Web Publishing',
       'Education', 'Software Training', 'Utilities',
       'Design &amp; Illustration', 'Audio Production', 'Photo Editing',
       'Accounting'], dtype=object)

In [9]:
# Reemplazar los valores en la columna 'genres' después de descodificar HTML
games_df['genres'] = games_df['genres'].apply(lambda x: html.unescape(x))
games_df['genres'] = games_df['genres'].str.replace("'Animation & Modeling'", 'Animation & Modeling')
games_df['genres'] = games_df['genres'].str.replace("'Design & Illustration'", 'Design & Illustration')

In [10]:
games_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71207 entries, 0 to 71206
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      71207 non-null  int64 
 1   year    71207 non-null  int32 
 2   genres  71207 non-null  object
dtypes: int32(1), int64(1), object(1)
memory usage: 1.4+ MB


In [11]:
games_df.shape

(71207, 3)

Ahora cargaremos el segundo dataframe.

In [12]:
items_df = pd.read_csv("./Datasets/user_items_procesado.csv")
items_df = items_df[["item_id", "playtime_forever"]]

items_df.head(3)

,item_id,playtime_forever
0,10,0.10
1,20,0.00
2,30,0.12


In [13]:
items_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5094092 entries, 0 to 5094091
Data columns (total 2 columns):
 #   Column            Dtype  
---  ------            -----  
 0   item_id           int64  
 1   playtime_forever  float64
dtypes: float64(1), int64(1)
memory usage: 77.7 MB


In [14]:
items_df.isna().sum() # No hay valores nulos

item_id             0
playtime_forever    0
dtype: int64

Vamos a sumar las horas jugadas para cada juego.

In [15]:
total_hours_by_game = items_df.groupby("item_id").playtime_forever.sum()
total_hours_by_game

item_id
10        285131.65
20         16009.26
30         12606.34
40          2573.98
50         12109.32
            ...    
528580         0.00
528660         3.03
529670         0.00
529820         0.00
530720         0.00
Name: playtime_forever, Length: 10978, dtype: float64

Convertimos la serie anterior a un dataframe.

In [16]:
items_df = total_hours_by_game.to_frame().reset_index()
items_df.head(3)

,item_id,playtime_forever
0,10,285131.65
1,20,16009.26
2,30,12606.34


Procedemos a hacer un inner join de ambos dataframes.

In [17]:
PlayTimeGenre_df = games_df.merge(items_df, left_on='id', right_on='item_id', 
                                  how='inner')

In [18]:
PlayTimeGenre_df.head()

,id,year,genres,item_id,playtime_forever
0,282010,1997,Action,282010,155.32
1,282010,1997,Indie,282010,155.32
2,282010,1997,Racing,282010,155.32
3,70,1998,Action,70,44182.89
4,1640,2006,Strategy,1640,452.71


Podemos eliminar las columnas "id" e "item_id".

In [19]:
PlayTimeGenre_df.drop(["id", "item_id"], axis=1, inplace=True)

Ahora agruparemos por género y por año y sumaremos las horas de juego totales.

In [20]:
PlayTimeGenre_df = PlayTimeGenre_df.groupby(["genres", "year"]).playtime_forever.sum()
PlayTimeGenre_df

genres          year
Action          1983      57.92
                1984       6.40
                1988     266.68
                1989      10.11
                1990     305.63
                         ...   
Web Publishing  2013    5561.50
                2014     560.80
                2015    5811.27
                2016       2.27
                2017     156.37
Name: playtime_forever, Length: 343, dtype: float64

In [21]:
PlayTimeGenre_df = PlayTimeGenre_df.unstack(0)
PlayTimeGenre_df.head()

genres,Action,Adventure,Animation & Modeling,Audio Production,Casual,Design & Illustration,Early Access,Education,Free to Play,Indie,...,Photo Editing,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
year,,,,,,,,,,,,,,,,,,,,,
1983,57.92,57.92,NaN,NaN,57.92,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1984,6.40,6.40,NaN,NaN,6.40,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1987,NaN,128.57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.92,NaN,NaN,NaN,NaN,NaN,NaN
1988,266.68,499.44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,232.76,...,NaN,232.76,NaN,0.02,NaN,NaN,232.76,NaN,NaN,NaN
1989,10.11,155.23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,84.74,NaN,NaN,NaN,NaN,NaN,NaN


Procederemos a guardar este dataframe en un .csv para ser consultado por la API.

In [22]:
PlayTimeGenre_df.to_csv("./Datasets_API/PlayTimeGenre.csv")

## UserForGenre

**Consigna:**

def UserForGenre(genero : str): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.

Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

## sentiment_analysis

**Consigna:**

def sentiment_analysis(año : int): Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.

Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}

Algunas consideraciones:
* El **análisis de sentimientos** está en la columna "sentiment_analysis" del dataset "user_reviews_procesado.csv".
* El **año de lanzamiento** está en la columna "year" del dataset "steam_games_procesado.csv".
* Podemos vincular ambos datasets a través de la columna **"id"** de "steam_games_procesado.csv" y la columna **"item_id"** de "user_reviews_procesado.csv".

Cargamos el dataset de steam_games.

In [23]:
games_df = pd.read_csv("./Datasets/steam_games_procesado.csv")
games_df = games_df[["id", "year"]]

games_df.head()

,id,year
0,761140,2018.0
1,643980,2018.0
2,670290,2017.0
3,767400,2017.0
4,773570,NaN


In [24]:
# Analizar los valores nulos.
games_df.isna().sum()

id         0
year    2170
dtype: int64

Para esta consulta no nos sirven aquellos registros que tengan valores nulos en "year" o "genres", por lo que los eliminaremos.

In [25]:
# Borramos valores nulos
games_df.dropna(subset=["year"], inplace=True, ignore_index=True)

# Casteamos "year" a entero
games_df["year"] = games_df["year"].astype("int32")

In [26]:
games_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29955 entries, 0 to 29954
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   id      29955 non-null  int64
 1   year    29955 non-null  int32
dtypes: int32(1), int64(1)
memory usage: 351.2 KB


Cargamos el dataset de user_reviews.

In [27]:
reviews_df = pd.read_csv("./Datasets/user_reviews_procesado.csv")
reviews_df = reviews_df[["item_id", "sentiment_analysis"]]

reviews_df.head()

,item_id,sentiment_analysis
0,1250,2
1,22200,2
2,43110,2
3,251610,2
4,227300,0


In [28]:
# Analizar los valores nulos.
reviews_df.isna().sum()

item_id               0
sentiment_analysis    0
dtype: int64

In [29]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58431 entries, 0 to 58430
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   item_id             58431 non-null  int64
 1   sentiment_analysis  58431 non-null  int64
dtypes: int64(2)
memory usage: 913.1 KB


Para saber cuántos sentimientos negativos (0), neutrales (1) y positivos (2) hay por cada videojuego, tendremos que hacer one-hot encoding en la columna "sentiment_analysis".

In [30]:
# Crear variables dummies para "sentiment_analysis".
sentiment_dummies = pd.get_dummies(reviews_df.sentiment_analysis).astype(int)

# Concatenar reviews_df con sentiment_dummies.
reviews_df = pd.concat([reviews_df, sentiment_dummies], axis = 1)

# Eliminar la columna "sentiment_analysis".
reviews_df.drop("sentiment_analysis", axis=1, inplace=True)

reviews_df.head()

,item_id,0,1,2
0,1250,0,0,1
1,22200,0,0,1
2,43110,0,0,1
3,251610,0,0,1
4,227300,1,0,0


Vamos a agrupar por item_id para saber el número de reseñas para cada juego.

In [31]:
reviews_df = reviews_df.groupby("item_id").sum()
reviews_df.head(3)

,0,1,2
item_id,,,
10,6,23,27
20,4,5,8
30,1,1,2


In [32]:
# Resetear el indice para que "item_id" pase a ser una columna.
reviews_df.reset_index(inplace=True)
reviews_df.head(3)

,item_id,0,1,2
0,10,6,23,27
1,20,4,5,8
2,30,1,1,2


Procedemos a hacer un inner join de ambos dataframes.

In [33]:
sentiment_analysis_df = games_df.merge(reviews_df, left_on='id', right_on='item_id', 
                                  how='inner')

In [34]:
sentiment_analysis_df.head(3)

,id,year,item_id,0,1,2
0,282010,1997,282010,0,1,0
1,70,1998,70,15,10,36
2,2400,2006,2400,6,5,12


Podemos eliminar las columnas "id" e "item_id".

In [35]:
sentiment_analysis_df.drop(["id", "item_id"], axis=1, inplace=True)

Ahora agruparemos por año y sumaremos el análisis de sentimientos de cada tipo.

In [36]:
sentiment_analysis_df = sentiment_analysis_df.groupby("year").sum()
sentiment_analysis_df.head(3)

,0,1,2
year,,,
1989,0,0,1
1990,1,0,4
1991,0,0,1


Procederemos a guardar este dataframe en un .csv para ser consultado por la API.

In [37]:
sentiment_analysis_df.to_csv("./Datasets_API/sentiment_analysis.csv")